In [ ]:
import os
import time
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import sounddevice as sd
import soundfile as sf
import numpy as np
import pandas as pd
import threading
import random
import datetime
import json
from pathlib import Path

# Configuration
BASE_DIR = r"C:\Users\cogpsy-vrlab\Documents\PPS_module\BreathingPilot"
PRACTICE_STIMULI_DIR = os.path.join(BASE_DIR, "PracticeStimuli")
PRACTICE_LOGS_DIR = os.path.join(PRACTICE_STIMULI_DIR, "logs")

# Response window in seconds (changed from 1.0 to 1.5)
RESPONSE_WINDOW = 1.5

# Ensure log directory exists
os.makedirs(PRACTICE_LOGS_DIR, exist_ok=True)

class PracticeTrialRunner:
    def __init__(self):
        # Initialize variables
        self.current_iteration = None
        self.played_iterations = set()
        self.available_iterations = set(range(1, 6))  # 5 different iterations
        self.experiment_running = False
        self.start_time = None
        self.mouse_clicks = []
        self.current_log_file = None
        self.practice_passed = False
        self.timeline_markers = []
        self.click_count = 0
        # Flag to control audio playback
        self.stop_audio = False
        
        # Load already played iterations from log
        self.load_played_iterations()
        
        # Create GUI
        self.create_gui()

    def update_progress_line(self, elapsed_time):
        """Update the position of the progress line based on elapsed time."""
        try:
            # Calculate position
            timeline_start_x = 400
            timeline_width = self.timeline_end_x - timeline_start_x
            max_duration = 90  # Maximum duration in seconds
            
            x_pos = timeline_start_x + min(elapsed_time / max_duration, 1.0) * timeline_width
            
            # Update line position
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y - 30, 
                x_pos, self.timeline_y + 30
            )
            return True
        except Exception as e:
            print(f"Error updating progress line: {e}")
            return False
    
    def load_played_iterations(self):
        """Load list of previously played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    self.played_iterations = set(json.load(f))
                print(f"Loaded previously played iterations: {self.played_iterations}")
                
                # Update available iterations
                self.available_iterations = set(range(1, 6)) - self.played_iterations
                if not self.available_iterations:
                    # Reset if all have been played
                    self.available_iterations = set(range(1, 6))
                    self.played_iterations = set()
        except Exception as e:
            print(f"Error loading played iterations: {e}")
    
    def save_played_iterations(self):
        """Save list of played iterations."""
        try:
            log_file = os.path.join(PRACTICE_LOGS_DIR, "played_iterations.json")
            with open(log_file, 'w') as f:
                json.dump(list(self.played_iterations), f)
        except Exception as e:
            print(f"Error saving played iterations: {e}")
    
    def create_gui(self):
        """Create the GUI."""
        self.root = tk.Tk()
        self.root.title("PPS Practice Trials")
        
        # Set protocol for window close
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        
        # Get screen dimensions
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        
        # Set window to 90% of screen size to account for taskbar and window decorations
        window_width = int(screen_width * 0.9)
        window_height = int(screen_height * 0.9)
        self.root.geometry(f"{window_width}x{window_height}+{int(screen_width*0.05)}+{int(screen_height*0.05)}")
        
        # Allow window resizing
        self.root.resizable(True, True)
        
        # Create main canvas with scrollbar
        self.main_canvas = tk.Canvas(self.root)
        self.main_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Add vertical scrollbar to canvas
        self.vsb = ttk.Scrollbar(self.root, orient=tk.VERTICAL, command=self.main_canvas.yview)
        self.vsb.pack(side=tk.RIGHT, fill=tk.Y)
        self.main_canvas.configure(yscrollcommand=self.vsb.set)
        
        # Create a frame inside the canvas for all content
        self.main_frame = ttk.Frame(self.main_canvas, padding="10")
        self.main_canvas.create_window((0, 0), window=self.main_frame, anchor=tk.NW, tags="self.main_frame")
        
        # Configure the canvas to resize with the frame
        self.main_frame.bind("<Configure>", lambda e: self.main_canvas.configure(scrollregion=self.main_canvas.bbox("all")))
        
        # Bind mousewheel to scroll
        self.root.bind("<MouseWheel>", lambda e: self.main_canvas.yview_scroll(int(-1*(e.delta/120)), "units"))  # For Windows
        self.root.bind("<Button-4>", lambda e: self.main_canvas.yview_scroll(-1, "units"))  # For Linux
        self.root.bind("<Button-5>", lambda e: self.main_canvas.yview_scroll(1, "units"))   # For Linux
        
        # Initialize all widgets that will be referenced later
        self.continue_button = None
        self.show_diagnostic_button = None
        self.try_again_button = None
        self.start_button = None
        self.timeline_markers = []
        
        # Bind mouse clicks for the entire application - do this FIRST
        self.root.bind("<Button-1>", self.on_mouse_click)
        print("Mouse click binding established")
        
        # Header
        ttk.Label(self.main_frame, text="PPS Practice Trials", 
                  font=("Arial", 16, "bold")).pack(pady=5)
        
        # Instructions frame - more compact
        instructions_frame = ttk.LabelFrame(self.main_frame, text="Instructions", padding=5)
        instructions_frame.pack(fill=tk.X, padx=5, pady=5)
        
        instruction_text = (
            "1. Click 'Start Trial' to begin a practice trial\n"
            "2. Listen carefully for the stimuli\n"
            "3. Click the mouse button ONLY when you hear a tactile stimulus (vibration sound)\n"
            f"4. You must click within {RESPONSE_WINDOW} seconds after hearing the tactile stimulus\n"
            "5. DO NOT click when you only hear a looming sound without tactile stimulus\n"
            "6. The 'Continue to Experiment' button will turn green when you're ready"
        )
        
        ttk.Label(instructions_frame, text=instruction_text, 
                  font=("Arial", 10), justify="left").pack(anchor=tk.W, pady=2)
        
        # Status frame - more compact
        self.status_frame = ttk.Frame(self.main_frame)
        self.status_frame.pack(fill=tk.X, padx=5, pady=2)
        
        self.status_var = tk.StringVar(value="Ready to start practice")
        self.status_label = ttk.Label(self.status_frame, textvariable=self.status_var, 
                                      font=("Arial", 11, "bold"))
        self.status_label.pack(pady=2)
        
        # Experimenter visualization frame
        visual_frame = ttk.LabelFrame(self.main_frame, text="Experimenter Visualization", padding=5)
        visual_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create canvas for visual indicators - reduced height
        self.visual_canvas = tk.Canvas(visual_frame, width=window_width - 100, height=100, bg="white")
        self.visual_canvas.pack(fill=tk.X, pady=2)
        
        # Create indicators
        self.tactile_indicator = self.visual_canvas.create_rectangle(50, 20, 350, 45, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 32, text="Tactile Stimulus", font=("Arial", 9))
        
        self.click_indicator = self.visual_canvas.create_rectangle(50, 55, 350, 80, fill="lightgray", outline="black")
        self.visual_canvas.create_text(200, 67, text="Mouse Click", font=("Arial", 9))
        
        # Add click counter display
        self.click_counter_text = self.visual_canvas.create_text(
            650, 67,
            text="Total Clicks: 0",
            font=("Arial", 11, "bold"),
            fill="blue"
        )
        
        # Timeline - make it span almost the entire width
        timeline_start_x = 400
        self.timeline_end_x = window_width - 120
        self.timeline_y = 75
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y, self.timeline_end_x, self.timeline_y, width=2)
        self.visual_canvas.create_line(timeline_start_x, self.timeline_y - 20, self.timeline_end_x, self.timeline_y - 20, width=2)
        
        # Add labels to the timeline
        self.visual_canvas.create_text(timeline_start_x - 30, self.timeline_y, text="Timeline:", font=("Arial", 9, "bold"))
        
        # Add time markers every 10 seconds
        for sec in range(0, 91, 10):
            x_pos = timeline_start_x + ((self.timeline_end_x - timeline_start_x) * sec / 90)
            self.visual_canvas.create_line(x_pos, self.timeline_y - 5, x_pos, self.timeline_y + 5, width=1)
            self.visual_canvas.create_text(x_pos, self.timeline_y + 12, text=f"{sec}s", font=("Arial", 7))
        
        # Create progress indicator (vertical line) - represents current playback position
        self.progress_line = self.visual_canvas.create_line(
            timeline_start_x, self.timeline_y - 20, timeline_start_x, self.timeline_y + 20, 
            width=3, fill="green", state="hidden"
        )
            
        # Create a separate frame for the click test area - reduced height
        click_test_frame = ttk.LabelFrame(self.main_frame, text="Click Test Area", padding=5)
        click_test_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create a canvas for the click test area - reduced height
        test_area_height = 120
        self.click_test_canvas = tk.Canvas(click_test_frame, width=window_width - 100, height=test_area_height, bg="lightyellow")
        self.click_test_canvas.pack(fill=tk.X, pady=2)
        
        # Add text to the click test area
        self.click_test_canvas.create_text(
            (window_width - 100)//2, test_area_height//2,
            text="CLICK ANYWHERE IN THIS AREA TO VERIFY MOUSE TRACKING",
            font=("Arial", 14, "bold"), fill="blue"
        )
            
        # Add a visual indicator for clicks in the test area
        self.click_feedback_circle = self.click_test_canvas.create_oval(
            (window_width - 100)//2 - 40, test_area_height//2 - 40,
            (window_width - 100)//2 + 40, test_area_height//2 + 40,
            outline="gray", width=2, fill="white"
        )
        
        # Results frame
        self.results_frame = ttk.LabelFrame(self.main_frame, text="Results", padding=5)
        self.results_var = tk.StringVar(value="Complete a practice trial to see results")
        self.results_label = ttk.Label(self.results_frame, textvariable=self.results_var, 
                                       font=("Arial", 10), justify="left")
        self.results_label.pack(anchor=tk.W, pady=2)
        self.results_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Button styles
        self.continue_style = ttk.Style()
        self.continue_style.configure("Red.TButton", background="lightgray")
        self.continue_style.configure("Green.TButton", background="green")
        self.continue_style.configure("Start.TButton", font=("Arial", 11, "bold"))
        
        # Create a distinctive frame for the Start button
        start_frame = ttk.LabelFrame(self.main_frame, text="Begin Practice", padding=5)
        start_frame.pack(fill=tk.X, padx=5, pady=2)
        
        # Create a horizontal frame for the buttons
        buttons_horizontal = ttk.Frame(start_frame)
        buttons_horizontal.pack(fill=tk.X, pady=5)
        
        # Start Trial button and Try Again button side by side
        self.start_button = ttk.Button(
            buttons_horizontal, text="START TRIAL",
            command=self.start_trial, width=25, style="Start.TButton"
        )
        self.start_button.pack(side=tk.LEFT, padx=5)
        
        self.try_again_button = ttk.Button(
            buttons_horizontal, text="TRY AGAIN", 
            command=self.start_trial, width=25, style="Start.TButton"
        )
        self.try_again_button.pack(side=tk.RIGHT, padx=5)
        self.try_again_button.pack_forget()  # Hide initially
        
        # Add Diagnostic Report button
        self.show_diagnostic_button = ttk.Button(
            self.results_frame, text="Show Detailed Diagnostics",
            command=self.show_diagnostic_report, state=tk.DISABLED
        )
        self.show_diagnostic_button.pack(anchor=tk.E, pady=2)
        
        # Bottom button frame
        self.button_frame = ttk.Frame(self.main_frame)
        self.button_frame.pack(pady=5)
        
        # Continue and Cancel buttons side by side
        self.continue_button = ttk.Button(
            self.button_frame, text="Continue to Experiment",
            state=tk.DISABLED, style="Red.TButton", width=20,
            command=lambda: messagebox.showinfo("Continue", "Proceeding to main experiment...")
        )
        self.continue_button.pack(side=tk.LEFT, padx=5)
        
        self.cancel_button = ttk.Button(
            self.button_frame, text="Cancel",
            command=self.on_close, width=15
        )
        self.cancel_button.pack(side=tk.LEFT, padx=5)
        
        # Ensure scrolling works properly by updating the scroll region
        self.main_frame.update_idletasks()
        self.main_canvas.config(scrollregion=self.main_canvas.bbox("all"))
        
        # Scroll to top when starting
        self.main_canvas.yview_moveto(0)

    def show_diagnostic_report(self):
        """Show detailed diagnostic report in a new window."""
        if not hasattr(self, 'diagnostic_report'):
            return
            
        # Create new window for report that fits screen better
        report_window = tk.Toplevel(self.root)
        report_window.title("Diagnostic Report")
        
        # Set size relative to screen
        window_width = min(800, self.root.winfo_screenwidth() - 100)
        window_height = min(600, self.root.winfo_screenheight() - 100)
        report_window.geometry(f"{window_width}x{window_height}")
        
        # Add scrolled text widget instead of manual scrollbar
        text_widget = scrolledtext.ScrolledText(
            report_window, wrap=tk.WORD, width=80, height=25
        )
        text_widget.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Insert report text
        text_widget.insert(tk.END, self.diagnostic_report)
        text_widget.config(state=tk.DISABLED)  # Make read-only
    
    def on_close(self):
        """Handle window close event."""
        if self.experiment_running:
            if messagebox.askyesno("Quit", "Practice trial is running. Are you sure you want to quit?"):
                # Stop audio playback when closing the window
                self.stop_audio = True
                sd.stop()
                print("Audio playback stopped")
                self.root.destroy()
        else:
            self.root.destroy()
    
    def on_mouse_click(self, event):
        """Handle mouse click events during the experiment."""
        print(f"Mouse click detected at screen position ({event.x}, {event.y})")
        
        # Show visual feedback on click test canvas if it exists
        try:
            if hasattr(self, 'click_test_canvas') and self.click_test_canvas.winfo_exists():
                # Check if click is within the click test canvas
                test_canvas_x = self.click_test_canvas.winfo_rootx()
                test_canvas_y = self.click_test_canvas.winfo_rooty()
                test_canvas_width = self.click_test_canvas.winfo_width()
                test_canvas_height = self.click_test_canvas.winfo_height()
                
                # If click is within the test canvas
                if (test_canvas_x <= event.x_root <= test_canvas_x + test_canvas_width and
                    test_canvas_y <= event.y_root <= test_canvas_y + test_canvas_height):
                    # Convert to canvas coordinates
                    canvas_x = event.x_root - test_canvas_x
                    canvas_y = event.y_root - test_canvas_y
                    
                    # Move the feedback circle to click position
                    self.click_test_canvas.coords(
                        self.click_feedback_circle,
                        canvas_x - 25, canvas_y - 25,
                        canvas_x + 25, canvas_y + 25
                    )
                    # Change color to indicate click
                    self.click_test_canvas.itemconfig(self.click_feedback_circle, fill="orange", outline="red")
                    # Reset after a short delay
                    self.root.after(300, lambda: self.click_test_canvas.itemconfig(
                        self.click_feedback_circle, fill="white", outline="gray"))
        except Exception as e:
            print(f"Error updating click test feedback: {e}")
        
        # Always show click visually on the main canvas
        try:
            if event.widget == self.visual_canvas:
                # Create a temporary visual flash where the click happened
                flash = self.visual_canvas.create_oval(event.x-10, event.y-10, event.x+10, event.y+10, 
                                                       fill="orange", outline="red", width=2)
                self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
        except Exception as e:
            print(f"Error creating click flash: {e}")
            
        if not self.experiment_running or self.start_time is None:
            print("Click ignored for scoring (experiment not running)")
            return  # Ignore clicks when not running
            
        # Calculate time since experiment start
        current_time = time.perf_counter() - self.start_time
        
        # Add to mouse clicks list
        self.mouse_clicks.append({
            "time": current_time,
            "timestamp": datetime.datetime.now().isoformat(),
            "x": event.x,
            "y": event.y
        })
        
        # Update click count
        self.click_count += 1
        try:
            self.visual_canvas.itemconfig(
                self.click_counter_text, 
                text=f"Total Clicks: {self.click_count}"
            )
        except Exception as e:
            print(f"Error updating click counter: {e}")
        
        # Print to console for debugging
        print(f"Mouse click at {current_time:.3f} seconds (WILL BE SCORED)")
        
        # Visual indicator for the experimenter
        self.flash_indicator(self.click_indicator, "green")
        
        # Add click marker to timeline
        try:
            self.add_timeline_marker(current_time, "red")
            print(f"Added timeline marker at {current_time:.3f}s")
        except Exception as e:
            print(f"Error adding timeline marker: {e}")
    
    def flash_indicator(self, indicator, color, duration=0.5):
        """Flash an indicator on the canvas for the experimenter."""
        try:
            original_color = self.visual_canvas.itemcget(indicator, "fill")
            self.visual_canvas.itemconfig(indicator, fill=color)
            self.root.update_idletasks()
            
            # Schedule color change back
            def reset_color():
                try:
                    if self.visual_canvas.winfo_exists():
                        self.visual_canvas.itemconfig(indicator, fill=original_color)
                except Exception as e:
                    print(f"Error resetting indicator color: {e}")
            
            self.root.after(int(duration * 1000), reset_color)
            return True
        except Exception as e:
            print(f"Error flashing indicator: {e}")
            return False
    
    def add_timeline_marker(self, time_sec, color):
        """Add a marker to the timeline at the specified time."""
        timeline_start_x = 400
        timeline_width = self.timeline_end_x - timeline_start_x
        
        # Calculate position based on time
        max_duration = 90  # Maximum duration in seconds
        x_pos = timeline_start_x + min(time_sec / max_duration, 1.0) * timeline_width
        
        try:
            # Create marker
            marker = self.visual_canvas.create_oval(
                x_pos-5, self.timeline_y-5, x_pos+5, self.timeline_y+5, 
                fill=color, outline="black", width=1
            )
            self.timeline_markers.append(marker)
            
            # Flash the timeline area briefly
            flash = self.visual_canvas.create_rectangle(
                x_pos-10, self.timeline_y-10, x_pos+10, self.timeline_y+10, 
                outline="red", width=2
            )
            self.root.after(200, lambda f=flash: self.visual_canvas.delete(f))
            
            # Update progress line
            self.visual_canvas.itemconfig(self.progress_line, state="normal")
            self.visual_canvas.coords(
                self.progress_line, 
                x_pos, self.timeline_y-20, 
                x_pos, self.timeline_y+20
            )
            
            return True
        except Exception as e:
            print(f"Error creating timeline marker: {e}")
            return False
    
    def clear_timeline(self):
        """Clear all markers from the timeline."""
        print("Clearing timeline markers...")
        try:
            for marker in self.timeline_markers:
                self.visual_canvas.delete(marker)
            self.timeline_markers = []
            
            # Add a flash effect to show timeline has been cleared
            flash = self.visual_canvas.create_rectangle(395, 15, 745, 95, outline="blue", width=2)
            self.root.after(300, lambda f=flash: self.visual_canvas.delete(f))
            print("Timeline cleared successfully")
        except Exception as e:
            print(f"Error clearing timeline: {e}")
    
    def select_next_iteration(self):
        """Select the next iteration that hasn't been played yet."""
        if not self.available_iterations:
            # Reset if all have been played
            self.available_iterations = set(range(1, 6))
            self.played_iterations = set()
        
        next_iteration = random.choice(list(self.available_iterations))
        self.available_iterations.remove(next_iteration)
        self.played_iterations.add(next_iteration)
        self.save_played_iterations()
        
        return next_iteration
    
    def start_trial(self):
        """Start a practice trial."""
        print("\n===== STARTING PRACTICE TRIAL =====")
        
        # Reset stop audio flag
        self.stop_audio = False
        
        # Reset click counter for new trial
        self.click_count = 0
        self.visual_canvas.itemconfig(self.click_counter_text, text=f"Total Clicks: {self.click_count}")
        
        # Disable buttons during trial
        self.start_button.config(state=tk.DISABLED)
        self.try_again_button.config(state=tk.DISABLED)
        self.continue_button.config(state=tk.DISABLED)
        self.show_diagnostic_button.config(state=tk.DISABLED)
        
        # Reset variables
        self.mouse_clicks = []
        self.experiment_running = True
        
        # Reset indicators
        self.visual_canvas.itemconfig(self.tactile_indicator, fill="lightgray")
        self.visual_canvas.itemconfig(self.click_indicator, fill="lightgray")
        
        # Hide progress line initially
        self.visual_canvas.itemconfig(self.progress_line, state="hidden")
        
        # Select next iteration
        self.current_iteration = self.select_next_iteration()
        print(f"Selected iteration {self.current_iteration}")
        
        # Update status
        self.status_var.set(f"Starting practice trial (Iteration {self.current_iteration})...")
        self.root.update()
        
        # Create log file
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_log_file = os.path.join(PRACTICE_LOGS_DIR, f"practice_log_iteration_{self.current_iteration}_{timestamp}.csv")
        
        # Verify mouse tracking before starting trial
        verify_msg = "Verifying mouse tracking..."
        print(verify_msg)
        self.status_var.set(verify_msg)
        self.root.update()
        
        # Flash the click test canvas to draw attention
        test_canvas_width = self.click_test_canvas.winfo_width()
        test_canvas_height = self.click_test_canvas.winfo_height()
        
        # Add a flashing rectangle over the entire test canvas
        self.flash_area_rect = self.click_test_canvas.create_rectangle(
            5, 5, test_canvas_width - 5, test_canvas_height - 5,
            outline="red", width=5
        )
            
        # Add text overlay
        self.flash_area_text = self.click_test_canvas.create_text(
            test_canvas_width//2, test_canvas_height//2,
            text="CLICK ANYWHERE TO VERIFY MOUSE TRACKING!",
            font=("Arial", 14, "bold"), fill="red"
        )
        
        # Wait a moment for user to notice
        self.root.after(2000, self.remove_verification_prompt)
        
        # Start trial in separate thread
        threading.Thread(target=self.run_trial, daemon=True).start()
    
    def remove_verification_prompt(self):
        """Remove the verification prompt and continue with trial."""
        try:
            self.click_test_canvas.delete(self.flash_area_rect)
            self.click_test_canvas.delete(self.flash_area_text)
            print("Verification prompt removed, continuing with trial")
        except Exception as e:
            print(f"Error removing verification prompt: {e}")
    
    def run_trial(self):
        """Run the actual practice trial."""
        try:
            # Clear previous timeline markers
            self.clear_timeline()
            
            # Find audio files for the current iteration
            looming_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_looming.wav")
            tactile_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_tactile.wav")
            log_file = os.path.join(PRACTICE_STIMULI_DIR, f"practice_iteration_{self.current_iteration}_log.csv")
            
            print(f"Looking for files:")
            print(f"- Looming: {looming_file}")
            print(f"- Tactile: {tactile_file}")
            print(f"- Log: {log_file}")
            
            # Check if files exist
            if not os.path.exists(looming_file) or not os.path.exists(tactile_file) or not os.path.exists(log_file):
                self.status_var.set(f"Error: Audio files for iteration {self.current_iteration} not found")
                print(f"ERROR: Could not find one or more files for iteration {self.current_iteration}")
                self.start_button.config(state=tk.NORMAL)
                self.experiment_running = False
                return
            
            # Load the trial log to know when tactile stimuli occur
            trial_log = pd.read_csv(log_file)
            print(f"Loaded trial log with {len(trial_log)} trials")
            print(trial_log[['trial_number', 'trial_type', 'tactile_time_seconds']])
            
            # Filter for trials with tactile stimuli (baseline and pps types)
            tactile_trials = trial_log[trial_log['trial_type'].isin(['baseline', 'pps'])]
            tactile_times = tactile_trials['tactile_time_seconds'].values
            print(f"Found {len(tactile_times)} tactile stimulus events at: {tactile_times}")
            
            # Add tactile event markers to timeline
            for t_time in tactile_times:
                try:
                    self.add_timeline_marker(t_time, "blue")
                except Exception as e:
                    print(f"Error adding tactile marker: {e}")
            
            # Load audio files
            looming_data, looming_sr = sf.read(looming_file)
            tactile_data, tactile_sr = sf.read(tactile_file)
            
            print(f"Loaded audio files:")
            print(f"- Looming: {len(looming_data)/looming_sr:.2f}s, {looming_sr}Hz, shape: {looming_data.shape}")
            print(f"- Tactile: {len(tactile_data)/tactile_sr:.2f}s, {tactile_sr}Hz, shape: {tactile_data.shape}")
            
            # Convert stereo to mono if needed
            if len(looming_data.shape) > 1 and looming_data.shape[1] > 1:
                print("Converting looming audio from stereo to mono for playback")
                looming_data = np.mean(looming_data, axis=1)
            if len(tactile_data.shape) > 1 and tactile_data.shape[1] > 1:
                print("Converting tactile audio from stereo to mono for playback")
                tactile_data = np.mean(tactile_data, axis=1)
            
            def update_status(msg):
                try:
                    self.status_var.set(msg)
                    self.root.update_idletasks()
                except Exception as e:
                    print(f"Error updating status: {e}")
            
            self.root.after(0, update_status, "Practice trial running - click when you hear a tactile stimulus")
            
            # Set start time just before playback
            self.start_time = time.perf_counter()
            print(f"Starting audio playback at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
            
            # Reset progress line to start position
            self.update_progress_line(0)
            
            # Start monitor thread for tactile indicators
            tactile_monitor_thread = threading.Thread(
                target=self.monitor_tactile_events, 
                args=(tactile_times,), 
                daemon=True
            )
            tactile_monitor_thread.start()
            
            # Play both audio files in separate threads with checks for the stop flag
            def play_looming():
                try:
                    with sd.OutputStream(samplerate=looming_sr, channels=1 if len(looming_data.shape) == 1 else looming_data.shape[1]) as stream:
                        chunk_size = 1024
                        for i in range(0, len(looming_data), chunk_size):
                            if self.stop_audio:
                                print("Looming audio playback stopped")
                                break
                            chunk = looming_data[i:min(i+chunk_size, len(looming_data))]
                            stream.write(chunk.astype(np.float32))
                    print("Looming audio playback completed")
                except Exception as e:
                    print(f"Error playing looming audio: {e}")
            
            def play_tactile():
                try:
                    with sd.OutputStream(samplerate=tactile_sr, channels=1 if len(tactile_data.shape) == 1 else tactile_data.shape[1]) as stream:
                        chunk_size = 1024
                        for i in range(0, len(tactile_data), chunk_size):
                            if self.stop_audio:
                                print("Tactile audio playback stopped")
                                break
                            chunk = tactile_data[i:min(i+chunk_size, len(tactile_data))]
                            stream.write(chunk.astype(np.float32))
                    print("Tactile audio playback completed")
                except Exception as e:
                    print(f"Error playing tactile audio: {e}")
            
            looming_thread = threading.Thread(target=play_looming, daemon=True)
            tactile_thread = threading.Thread(target=play_tactile, daemon=True)
            
            looming_thread.start()
            tactile_thread.start()
            
            # Wait for audio to finish
            max_duration = max(len(looming_data) / looming_sr, len(tactile_data) / tactile_sr)
            end_time = self.start_time + max_duration + 1.0
            
            # Start a progress update thread
            progress_update_thread = threading.Thread(
                target=self.continuous_progress_update,
                args=(max_duration,),
                daemon=True
            )
            progress_update_thread.start()
            
            # Update status periodically
            while time.perf_counter() < end_time and not self.stop_audio and (looming_thread.is_alive() or tactile_thread.is_alive()):
                elapsed = time.perf_counter() - self.start_time
                self.root.after(0, update_status, f"Practice trial running - {elapsed:.1f}s / {max_duration:.1f}s")
                time.sleep(0.1)
            
            if not self.stop_audio:
                # Wait for threads to complete
                looming_thread.join(timeout=1.0)
                tactile_thread.join(timeout=1.0)
                
                print(f"Audio playback completed at {datetime.datetime.now().strftime('%H:%M:%S.%f')}")
                self.root.after(0, update_status, "Practice trial completed - analyzing results...")
                
                # Analyze results
                self.analyze_results(tactile_times)
            else:
                print("Audio playback was stopped by user")
                self.root.after(0, update_status, "Practice trial stopped")
            
        except Exception as e:
            print(f"ERROR during practice trial: {e}")
            import traceback
            traceback.print_exc()
            
            def show_error(msg):
                self.status_var.set(msg)
            self.root.after(0, show_error, f"Error during practice trial: {str(e)}")
            
        finally:
            self.experiment_running = False
            
            def enable_buttons():
                self.start_button.config(state=tk.NORMAL)
                self.try_again_button.config(state=tk.NORMAL)
            self.root.after(0, enable_buttons)
            self.save_log()
    
    def continuous_progress_update(self, max_duration):
        """Continuously update the progress line during the experiment."""
        try:
            update_interval = 0.05  # Update every 50ms
            
            while self.experiment_running and not self.stop_audio:
                if self.start_time is not None:
                    elapsed = time.perf_counter() - self.start_time
                    if elapsed > max_duration:
                        break
                    # Schedule an update to the progress line
                    self.root.after(0, lambda t=elapsed: self.update_progress_line(t))
                    time.sleep(update_interval)
                else:
                    time.sleep(0.1)
            print("Progress update thread completed")
        except Exception as e:
            print(f"Error in progress update thread: {e}")
    
    def monitor_tactile_events(self, tactile_times):
        """Monitor and visualize tactile events during playback."""
        if not tactile_times.size:
            return
        try:
            while self.experiment_running and not self.stop_audio:
                current_time = time.perf_counter() - self.start_time
                for t_time in tactile_times:
                    if abs(current_time - t_time) < 0.1:  # within 100ms
                        self.root.after(0, lambda: self.flash_indicator(self.tactile_indicator, "yellow"))
                        print(f"Tactile event detected at {current_time:.3f}s (expected {t_time:.3f}s)")
                        break
                time.sleep(0.05)
        except Exception as e:
            print(f"Error in tactile monitor: {e}")
    
    def analyze_results(self, tactile_times):
        """Analyze the results of the practice trial."""
        print("\n===== ANALYZING PRACTICE RESULTS =====")
        print(f"Tactile stimuli at: {tactile_times}")
        print(f"Mouse clicks at: {[click['time'] for click in self.mouse_clicks]}")
        
        # Initialize counters
        correct_clicks = 0
        false_alarms = 0
        missed_stimuli = 0
        
        # Initialize result details
        correct_details = []
        false_alarm_details = []
        missed_details = []
        
        # Mark each stimulus as not responded yet
        stimulus_responded = [False] * len(tactile_times)
        
        # Check each mouse click
        for click_idx, click in enumerate(self.mouse_clicks):
            click_time = click["time"]
            print(f"\nAnalyzing click {click_idx+1} at {click_time:.3f}s")
            
            # Initialize variables for finding the closest tactile stimulus
            closest_idx = -1
            closest_diff = float('inf')
            
            # Find the closest preceding tactile stimulus
            for i, stim_time in enumerate(tactile_times):
                time_diff = click_time - stim_time
                # Only consider tactile stimuli that preceded this click and are closest
                if 0 < time_diff < closest_diff:
                    closest_diff = time_diff
                    closest_idx = i
            
            # Determine if the click is a correct response or false alarm
            # Changed from 1.0 to 1.5 seconds window
            if closest_idx >= 0 and closest_diff <= RESPONSE_WINDOW:
                print(f"  ✓ CORRECT: Click at {click_time:.3f}s matches tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s)")
                correct_clicks += 1
                stimulus_responded[closest_idx] = True
                correct_details.append({
                    'click_time': click_time,
                    'tactile_time': tactile_times[closest_idx],
                    'difference': closest_diff
                })
            else:
                if closest_idx >= 0:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s is too late for tactile at {tactile_times[closest_idx]:.3f}s (diff: {closest_diff:.3f}s > {RESPONSE_WINDOW}s)")
                else:
                    print(f"  ✗ FALSE ALARM: Click at {click_time:.3f}s doesn't match any tactile stimulus")
                false_alarms += 1
                false_alarm_details.append({
                    'click_time': click_time,
                    'closest_tactile': tactile_times[closest_idx] if closest_idx >= 0 else None,
                    'difference': closest_diff if closest_idx >= 0 else None
                })
        
        # Check for missed stimuli (tactile events without click responses)
        for i, responded in enumerate(stimulus_responded):
            if not responded:
                print(f"✗ MISSED: Tactile at {tactile_times[i]:.3f}s had no response within {RESPONSE_WINDOW} seconds")
                missed_stimuli += 1
                missed_details.append({'tactile_time': tactile_times[i]})
        
        # Calculate hit rate
        total_tactile = len(tactile_times)
        hit_rate = (correct_clicks / total_tactile) if total_tactile > 0 else 0
        
        print(f"\nSummary: {correct_clicks}/{total_tactile} correct, {false_alarms} false alarms, {missed_stimuli} misses")
        
        # Determine if practice passed (80% correct and <= 1 false alarm)
        self.practice_passed = (hit_rate >= 0.8) and (false_alarms <= 1)
        print(f"Practice passed: {self.practice_passed}")
        
        # Create detailed diagnostic report
        diagnostic_report = f"===== DETAILED DIAGNOSTIC REPORT =====\n\n"
        diagnostic_report += f"RESPONSE WINDOW: {RESPONSE_WINDOW} seconds\n\n"
        
        # Add correct responses section
        diagnostic_report += f"CORRECT RESPONSES ({correct_clicks}):\n"
        if correct_details:
            for i, detail in enumerate(correct_details):
                diagnostic_report += (
                    f"  {i+1}. Click at {detail['click_time']:.3f}s matched tactile "
                    f"at {detail['tactile_time']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        # Add false alarms section
        diagnostic_report += f"\nFALSE ALARMS ({false_alarms}):\n"
        if false_alarm_details:
            for i, detail in enumerate(false_alarm_details):
                if detail['closest_tactile'] is not None:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s was too late for "
                        f"tactile at {detail['closest_tactile']:.3f}s (diff: {detail['difference']:.3f}s)\n"
                    )
                else:
                    diagnostic_report += (
                        f"  {i+1}. Click at {detail['click_time']:.3f}s didn't match any tactile stimulus\n"
                    )
        else:
            diagnostic_report += "  None\n"
        
        # Add missed stimuli section
        diagnostic_report += f"\nMISSED STIMULI ({missed_stimuli}):\n"
        if missed_details:
            for i, detail in enumerate(missed_details):
                diagnostic_report += (
                    f"  {i+1}. Tactile at {detail['tactile_time']:.3f}s had no response within {RESPONSE_WINDOW} seconds\n"
                )
        else:
            diagnostic_report += "  None\n"
        
        # Add conclusion
        diagnostic_report += "\nCONCLUSION:\n"
        diagnostic_report += f"  Hit rate: {hit_rate*100:.1f}% ({correct_clicks}/{total_tactile})\n"
        diagnostic_report += f"  Requirement: ≥80% hit rate and ≤1 false alarm\n"
        diagnostic_report += f"  Result: {'PASSED' if self.practice_passed else 'FAILED'}\n"
        
        self.diagnostic_report = diagnostic_report
        
        # Enhanced display of results in the GUI
        def update_ui():
            try:
                detailed_results = (
                    f"Practice Results:\n\n"
                    f"Correct responses: {correct_clicks} / {total_tactile} ({hit_rate*100:.1f}%)\n"
                    f"False alarms: {false_alarms}\n"
                    f"Missed stimuli: {missed_stimuli}\n\n"
                )
                
                # Add specific details about misses
                if missed_details:
                    detailed_results += "MISSED STIMULI:\n"
                    for i, detail in enumerate(missed_details[:3]):  # Show first 3 for brevity
                        detailed_results += f"- Tactile at {detail['tactile_time']:.1f}s had no response\n"
                    if len(missed_details) > 3:
                        detailed_results += f"  (and {len(missed_details) - 3} more...)\n"
                    detailed_results += "\n"
                
                # Add specific details about false alarms
                if false_alarm_details:
                    detailed_results += "FALSE ALARMS:\n"
                    for i, detail in enumerate(false_alarm_details[:3]):  # Show first 3 for brevity
                        if detail['closest_tactile'] is not None:
                            detailed_results += f"- Click at {detail['click_time']:.1f}s (too late, {detail['difference']:.1f}s after tactile)\n"
                        else:
                            detailed_results += f"- Click at {detail['click_time']:.1f}s (no corresponding tactile)\n"
                    if len(false_alarm_details) > 3:
                        detailed_results += f"  (and {len(false_alarm_details) - 3} more...)\n"
                    detailed_results += "\n"
                
                if self.practice_passed:
                    detailed_results += "PASSED! You understood the task correctly."
                    self.status_var.set("Practice passed successfully!")
                    self.continue_button.config(state=tk.NORMAL, style="Green.TButton")
                else:
                    detailed_results += (
                        "FAILED. Please try again and remember:\n"
                        f"- Click ONLY when you hear a tactile stimulus\n"
                        f"- You must click within {RESPONSE_WINDOW} seconds after the stimulus\n"
                        f"- Don't click for looming sounds without tactile stimuli"
                    )
                    self.status_var.set("Practice failed - try again")
                
                self.results_var.set(detailed_results)
                
                # Show Try Again button instead of Start
                self.start_button.pack_forget()
                self.try_again_button.pack(side=tk.LEFT, padx=5)
                
                # Enable diagnostic button
                self.show_diagnostic_button.config(state=tk.NORMAL)
                
                # Ensure buttons are visible by scrolling to bottom if needed
                self.main_canvas.yview_moveto(1.0)
            except Exception as e:
                print(f"Error updating UI with results: {e}")
        
        self.root.after(0, update_ui)
    
    def save_log(self):
        """Save the mouse click log to a CSV file."""
        if not self.mouse_clicks:
            return
        try:
            df = pd.DataFrame(self.mouse_clicks)
            df['iteration'] = self.current_iteration
            df['passed'] = self.practice_passed
            df.to_csv(self.current_log_file, index=False)
            print(f"Saved mouse click log to {self.current_log_file}")
        except Exception as e:
            print(f"Error saving log: {e}")

def main():
    app = PracticeTrialRunner()
    app.root.mainloop()

if __name__ == "__main__":
    main()